In [1]:
import csv
from RDFunctions import *
# Go to each of 912 records
# Get SI series for each record using each method
# Then select a threshold and see sensitivity, specificity and lead time and penalized lead time for each

In [2]:
def getSIseries(serial_id,dfOuranno,method,model):
    OUTLIERSD = 2.5
    LOWESSWINDOW = 3.0
    record_name = dfOuranno.loc[serial_id,'RecordNum'] 
    patient_id = dfOuranno.loc[serial_id,'PatientId']
    flname = './FinalRecords/'+str(record_name)+'n.hea'
    recname = './FinalRecords/'+str(record_name)+'n'       
    
    [samples,R,S,H,firstline] = getIndexOfRRSpO2HR(flname)
    rec =  wfdb.io.rdrecord(str(recname))
    xrr = rec.p_signal[:,R]
    xspo2 = rec.p_signal[:,S]
    xhr = rec.p_signal[:,H]
    TOTAL_LEN = len(xrr)
    t = np.arange(0,TOTAL_LEN,1)
    
    [xrrnew,trrnew] = outlierRejector(xrr,t,OUTLIERSD,default=15.0)
    [xspo2new,tspo2new] = outlierRejector(xspo2,t,OUTLIERSD, default=98.0)
    [xhrnew,thrnew] = outlierRejector(xhr,t,OUTLIERSD)
    
    zrrnew = applyLowess(xrrnew,trrnew,LOWESSWINDOW*60)
    zspo2new = applyLowess(xspo2new,tspo2new,LOWESSWINDOW*60)
    zhrnew = applyLowess(xhrnew,thrnew,LOWESSWINDOW*60)
    
    tnew = zrrnew[:,0]/60.0
    
    rr_loess = zrrnew[:,1]
    spo2_loess = zspo2new[:,1]
        
    bar_h = []
    
    if(method=='LR' or method=='LGB' or method=='MLP' or method=='SVM'):
        for ii in range(24*60,TOTAL_LEN,60):
            feature_array = getLongFeatures(rr_loess[(ii-24*60):ii],spo2_loess[(ii-24*60):ii])
            feature_array = np.array(feature_array[0:12],dtype=np.float32)
            predict = model.predict_proba(feature_array.reshape(1,-1))[0][1]
            bar_h.append(predict)
            
    if(method=='CNN' or method=='LSTM'):
        seg = np.zeros((1,1440,2),dtype=np.float64)
        for ii in range(24*60,TOTAL_LEN,60):
            seg[0,:,0] = (rr_loess[(ii-24*60):ii] - 25)/10
            seg[0,:,1] = (spo2_loess[(ii-24*60):ii] - 93)/10
            predict = model.predict(seg)[0,0]
            bar_h.append(predict)
        
    return bar_h

In [3]:
import pickle
annofile = r'RecordsAnno4.csv'
dfOuranno = pd.read_csv(annofile, encoding='iso-8859-1') 

In [18]:
# load logistic regression model from disk
import time
LR_model = pickle.load(open('logistic_model.sav', 'rb'))
SIsequences = []
start_time = time.time()
for i in range(6):
    print(i,end=" ")
    SIsequences.append(getSIseries(i,dfOuranno,"LR",LR_model))
end_time = time.time()
count = sum([len(e) for e in SIsequences])

total_time = end_time - start_time
print('\nNumber of runs %d'%count)
print('Total Time Seconds %f'%total_time)
print('Average Time %f'%float(total_time/count))

0 1 2 3 4 5 
Number of runs 92
Total Time Seconds 8.750280
Average Time 0.095112


In [19]:
import time
# load lgb model from disk
LGB_model = pickle.load(open('lgb_model.sav', 'rb'))
SIsequences = []
start_time = time.time()

for i in range(6):
    print(i,end=" ")
    SIsequences.append(getSIseries(i,dfOuranno,"LGB",LGB_model))

end_time = time.time()
count = sum([len(e) for e in SIsequences])
total_time = end_time - start_time
print('\nNumber of runs %d'%count)
print('Total Time Seconds %f'%total_time)
print('Average Time %f'%float(total_time/count))

0 1 2 3 4 5 
Number of runs 92
Total Time Seconds 9.796193
Average Time 0.106480


In [20]:
import time
MLP_model = pickle.load(open('mlp_model.sav', 'rb'))
SIsequences = []
start_time = time.time()
for i in range(6):
    print(i,end=" ")
    SIsequences.append(getSIseries(i,dfOuranno,"MLP",MLP_model))
end_time = time.time()
count = sum([len(e) for e in SIsequences])
total_time = end_time - start_time
print('\nNumber of runs %d'%count)
print('Total Time Seconds %f'%total_time)
print('Average Time %f'%float(total_time/count))

0 1 2 3 4 5 
Number of runs 92
Total Time Seconds 8.706151
Average Time 0.094632


In [21]:
import time
SVM_model = pickle.load(open('svm_model.sav', 'rb'))
SIsequences = []
start_time = time.time()
for i in range(6):
    print(i,end=" ")
    SIsequences.append(getSIseries(i,dfOuranno,"SVM",SVM_model))
end_time = time.time()
count = sum([len(e) for e in SIsequences])
total_time = end_time - start_time
print('\nNumber of runs %d'%count)
print('Total Time Seconds %f'%total_time)
print('Average Time %f'%float(total_time/count))

0 1 2 3 4 5 
Number of runs 92
Total Time Seconds 8.709614
Average Time 0.094670


In [22]:
import tensorflow.keras as ks

# load CNN model from disk
CNN_model = ks.models.load_model('QuantCNN.h5')
SIsequences = []
start_time = time.time()
for i in range(6):
    print(i,end=" ")
    SIsequences.append(getSIseries(i,dfOuranno,"CNN",CNN_model))
end_time = time.time()
count = sum([len(e) for e in SIsequences])
total_time = end_time - start_time
print('\nNumber of runs %d'%count)
print('Total Time Seconds %f'%total_time)
print('Average Time %f'%float(total_time/count))

0 1 2 3 4 5 
Number of runs 92
Total Time Seconds 12.588340
Average Time 0.136830


In [23]:
import tensorflow.keras as ks

# load CNN model from disk
LSTM_model = ks.models.load_model('QuantLSTM.h5')
SIsequences = []
start_time = time.time()
for i in range(6):
    print(i,end=" ")
    SIsequences.append(getSIseries(i,dfOuranno,"LSTM",LSTM_model))
end_time = time.time()
count = sum([len(e) for e in SIsequences])
total_time = end_time - start_time
print('\nNumber of runs %d'%count)
print('Total Time Seconds %f'%total_time)
print('Average Time %f'%float(total_time/count))

0 1 2 3 4 5 
Number of runs 92
Total Time Seconds 32.277431
Average Time 0.350842


In [ ]:
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.48                 Driver Version: 410.48                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Quadro P5000        Off  | 00000000:02:00.0  On |                  Off |
| 30%   44C    P8     7W / 180W |  15720MiB / 16271MiB |      1%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU       PID   Type   Process name                             Usage      |
|=============================================================================|
|    0      1101      G   /usr/lib/xorg/Xorg                           149MiB |
|    0      2081      G   compiz                                       169MiB |
|    0    127397      C   /home/ubuntu/anaconda3/bin/python          15397MiB |
+-----------------------------------------------------------------------------+


In [26]:
 import platform,socket,re,uuid,json,psutil

def getSystemInfo():
    try:
        info={}
        info['platform']=platform.system()
        info['platform-release']=platform.release()
        info['platform-version']=platform.version()
        info['architecture']=platform.machine()
        info['hostname']=socket.gethostname()
        info['ip-address']=socket.gethostbyname(socket.gethostname())
        info['mac-address']=':'.join(re.findall('..', '%012x' % uuid.getnode()))
        info['processor']=platform.processor()
        info['ram']=str(round(psutil.virtual_memory().total / (1024.0 **3)))+" GB"
        return json.dumps(info)
    except Exception as e:
        logging.exception(e)

getSystemInfo()

'{"platform": "Linux", "platform-release": "4.4.0-93-generic", "platform-version": "#116-Ubuntu SMP Fri Aug 11 21:17:51 UTC 2017", "architecture": "x86_64", "hostname": "ubuntu-HP-Z440-Workstation", "ip-address": "127.0.1.1", "mac-address": "b1:aa:7a:5b:5b:47", "processor": "x86_64", "ram": "31 GB"}'

In [31]:
# comparing size of models
import os
print("Logistic Regression Model Size (in bytes) : %s"%os.stat('logistic_model.sav').st_size)
print("Decision Tree Model Size (in bytes) : %s"%os.stat('lgb_model.sav').st_size)
print("Multi Layer Perceptron Model Size (in bytes) : %s"%os.stat('mlp_model.sav').st_size)
print("Support Vecotor Model Size (in bytes) : %s"%os.stat('svm_model.sav').st_size)
print("CNN Model Size (in bytes) : %s"%os.stat('QuantCNN.h5').st_size)
print("LSTM Model Size (in bytes) : %s"%os.stat('QuantLSTM.h5').st_size)

Logistic Regression Model Size (in bytes) : 924
Decision Tree Model Size (in bytes) : 67857
Multi Layer Perceptron Model Size (in bytes) : 268831
Support Vecotor Model Size (in bytes) : 50193
CNN Model Size (in bytes) : 61608
LSTM Model Size (in bytes) : 75808
